In [1]:
import torch
from torch import nn
import os

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar
from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger

# Not precize but faster
torch.set_float32_matmul_precision('medium')

In [2]:
from Data.DataModels.datasetFromSequences import SequenceDataset
from Data.DataModels.dataModule import DataModule
from Models.modelLSTM import LSTMModel
from Models.modelMAMBA import MAMBAModel
from Models.modelTransformer import TransformerModel
from Models.moduleAdamW import AdamWModule
from Data.preprocessingChangeInPrice import PreprocessChangeInPrice

In [3]:
pl.seed_everything(42)

Seed set to 42


42

# Hyperparameters

In [4]:
data = PreprocessChangeInPrice(
    csv_path = 'Data/CSV/WIKI-NVDA.csv',
    date_column = 'Date',
    column_to_shift = 'Close',
    use_scaler = True,
    sequence_length = 120,
    val_percentage = .1,
    test_percentage = .1
)

data.preprocess()

Getting the change of price in a day...


  0%|          | 0/4825 [00:00<?, ?it/s]

Getting the change of price in a day...


  0%|          | 0/4825 [00:00<?, ?it/s]

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend',
       'Split Ratio', 'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close',
       'Adj. Volume', 'Previous', 'Previous_Change', 'Future',
       'Future_Change'],
      dtype='object')
Transforming date column to day_of_week, day_of_month, week_of_year and year columns...


...

Current shape after transformation of date column:  (4825, 12)
Current train and test shapes:  (3860, 12) (483, 12)
Creating sequences from pandas DataFrame...


  0%|          | 0/3740 [00:00<?, ?it/s]

  0%|          | 0/362 [00:00<?, ?it/s]

  0%|          | 0/363 [00:00<?, ?it/s]

Train shape:  (120, 12)
Val shape:  (120, 12)
Test shape:  (120, 12)


In [5]:
N_EPOCHS = 8
BATCH_SIZE = 64
LEARNING_RATE = 0.0001
SEQUENCE_LENGTH = 120
N_FEATURES = data.train_sequences[0][0].shape[1]

# LSTM Model

In [6]:
module = AdamWModule(
    n_features = N_FEATURES,
    model = LSTMModel,
    criterion = nn.MSELoss(),
    learning_rate = LEARNING_RATE
)

predictor_name = 'stock'
model_name = 'lstm'

checkpoint_callback = ModelCheckpoint(
    dirpath = f'Checkpoints/{predictor_name}-price/best_models',
    filename = f'{model_name}-{predictor_name}-checkpoint',
    save_top_k = 3,
    monitor = 'val_loss',
    mode = 'min'
)

tensorboard_logger = TensorBoardLogger(
    save_dir = f'Checkpoints/{predictor_name}-price/train_logs',
    name = f'{model_name}-{predictor_name}-logs'
)

csv_logger = CSVLogger(
    save_dir = f'Checkpoints/{predictor_name}-price/train_logs',
    name = f'{model_name}-{predictor_name}-logs'
)

early_stopping_callback = EarlyStopping(monitor = ' val_loss', patience = 2)

trainerLSTM = pl.Trainer(
    logger = [tensorboard_logger, csv_logger],
    callbacks=[checkpoint_callback, TQDMProgressBar(refresh_rate=30)],
    max_epochs = N_EPOCHS,
    accelerator='cuda',
    devices=1
)

/home/agata/.local/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:208: Attribute 'criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion'])`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


# Transformer Model

In [7]:
module = AdamWModule(
    n_features = N_FEATURES,
    model = TransformerModel,
    criterion = nn.MSELoss(),
    learning_rate = LEARNING_RATE
)

predictor_name = 'stock'
model_name = 'transformer'

checkpoint_callback = ModelCheckpoint(
    dirpath = f'Checkpoints/{predictor_name}-price/best_models',
    filename = f'{model_name}-{predictor_name}-checkpoint',
    save_top_k = 3,
    monitor = 'val_loss',
    mode = 'min'
)

tensorboard_logger = TensorBoardLogger(
    save_dir = f'Checkpoints/{predictor_name}-price/train_logs',
    name = f'{model_name}-{predictor_name}-logs'
)

csv_logger = CSVLogger(
    save_dir = f'Checkpoints/{predictor_name}-price/train_logs',
    name = f'{model_name}-{predictor_name}-logs'
)

early_stopping_callback = EarlyStopping(monitor = ' val_loss', patience = 2)

trainerTransformer = pl.Trainer(
    logger = [tensorboard_logger, csv_logger],
    callbacks=[checkpoint_callback, TQDMProgressBar(refresh_rate=30)],
    max_epochs = N_EPOCHS,
    accelerator='cuda',
    devices=1
)

/home/agata/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


# MAMBA Model

In [8]:
module = AdamWModule(
    n_features = N_FEATURES,
    model = MAMBAModel,
    criterion = nn.MSELoss(),
    learning_rate = LEARNING_RATE
)

predictor_name = 'stock'
model_name = 'mamba'

checkpoint_callback = ModelCheckpoint(
    dirpath = f'Checkpoints/{predictor_name}-price/best_models',
    filename = f'{model_name}-{predictor_name}-checkpoint',
    save_top_k = 3,
    monitor = 'val_loss',
    mode = 'min'
)

tensorboard_logger = TensorBoardLogger(
    save_dir = f'Checkpoints/{predictor_name}-price/train_logs',
    name = f'{model_name}-{predictor_name}-logs'
)

csv_logger = CSVLogger(
    save_dir = f'Checkpoints/{predictor_name}-price/train_logs',
    name = f'{model_name}-{predictor_name}-logs'
)

early_stopping_callback = EarlyStopping(monitor = ' val_loss', patience = 2)

trainerMAMBA = pl.Trainer(
    logger = [tensorboard_logger, csv_logger],
    callbacks=[checkpoint_callback, TQDMProgressBar(refresh_rate=30)],
    max_epochs = N_EPOCHS,
    accelerator='cuda',
    devices=1
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [9]:
csv_list = os.listdir('Data/CSV')
csv_list = [k for k in csv_list if '.csv' in k.lower()]
csv_list = [k for k in csv_list if not k in ['Binance_BTCUSDT_2024_minute.csv',
                                             'WIKI-MSFT.csv',
                                             'WIKI-SBUX.csv']]

In [10]:
for csv_file in csv_list:
    data = PreprocessChangeInPrice(
        csv_path = 'Data/CSV/' + csv_file,
        date_column = 'Date',
        column_to_shift = 'Close',
        use_scaler = False,
        sequence_length = 120,
        val_percentage = .1,
        test_percentage = .1
    )

    data.preprocess()

    data_module = DataModule(
        train_sequences = data.train_sequences,
        val_sequences = data.val_sequences,
        test_sequences = data.test_sequences,
        dataset = SequenceDataset,
        batch_size = BATCH_SIZE
    )
    data_module.setup()

    trainerLSTM.fit(module, data_module)
    trainerTransformer.fit(module, data_module)
    trainerMAMBA.fit(module, data_module)

Getting the change of price in a day...


  0%|          | 0/1949 [00:00<?, ?it/s]

Getting the change of price in a day...


  0%|          | 0/1949 [00:00<?, ?it/s]

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend',
       'Split Ratio', 'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close',
       'Adj. Volume', 'Previous', 'Previous_Change', 'Future',
       'Future_Change'],
      dtype='object')
Transforming date column to day_of_week, day_of_month, week_of_year and year columns...


...

`Trainer.fit` stopped: `max_epochs=8` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params | Mode 
-------------------------------------------------
0 | model     | MAMBAModel | 194 K  | train
1 | criterion | MSELoss    | 0      | train
-------------------------------------------------
194 K     Trainable params
0         Non-trainable params
194 K     Total params
0.777     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=8` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params | Mode 
-------------------------------------------------
0 | model     | MAMBAModel | 194 K  | train
1 | criterion | MSELoss    | 0      | train
-------------------------------------------------
194 K     Trainable params
0         Non-trainable params
194 K     Total params
0.777     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=8` reached.
